<a href="https://colab.research.google.com/github/Desy16/walmart-stock/blob/main/walmart_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PROJET ANALYSE DE DONNÉES SPARK WALMART

# Exercice 1
from google.colab import files
myfile = files.upload()

Saving walmart_stock.csv to walmart_stock.csv


In [ ]:
# 1. Session spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("WalmartStock").getOrCreate()

In [ ]:
# 2. Chargement du fichier CSV
df = spark.read.csv("walmart_stock.csv", header=True, inferSchema=True)

In [ ]:
# 3. Noms des colonnes
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [ ]:
# 4. Schema
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [ ]:
# 5. Cinq premières colonnes
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [ ]:
# 6. Détails du DataFrame
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [ ]:
# 7. DataFrame avec la colonne HV_Ration
df2 = df.withColumn("HV_Ratio", df["High"]/df["Volume"])

In [ ]:
# 8. Jour avec le prix le plus élevé
df.orderBy(df["High"].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

In [ ]:
# 9. Moyenne de la colonne Close
from pyspark.sql.functions import mean
df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [ ]:
# 10. Valeurs maximale et minimale de la colonne Volume
from pyspark.sql.functions import max, min
df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [ ]:
# 11. Jours de la clôture inférieure à 60 dollars
from pyspark.sql.functions import count
df.filter(df["Close"] < 60).select(count("Date")).show()

+-----------+
|count(Date)|
+-----------+
|         81|
+-----------+



In [ ]:
# 12. Pourcentage de temps avec la hauteur supérieure à 80 dollars
(df.filter(df["High"] > 80).count() / df.count()) * 100

9.141494435612083

In [ ]:
# 13. Maximum de High par an
from pyspark.sql.functions import year
df.groupBy(year("Date")).max("High").show()

+----------+---------+
|year(Date)|max(High)|
+----------+---------+
|      2015|90.970001|
|      2013|81.370003|
|      2014|88.089996|
|      2012|77.599998|
|      2016|75.190002|
+----------+---------+



In [ ]:
# 14. Clôture moyenne pour chaque mois
from pyspark.sql.functions import month
df.groupBy(month("Date")).mean("Close").show()

+-----------+-----------------+
|month(Date)|       avg(Close)|
+-----------+-----------------+
|         12|72.84792478301885|
|          1|71.44801958415842|
|          6| 72.4953774245283|
|          3|71.77794377570092|
|          5|72.30971688679247|
|          9|72.18411785294116|
|          4|72.97361900952382|
|          8|73.02981855454546|
|          7|74.43971943925233|
|         10|71.57854545454543|
|         11| 72.1110893069307|
|          2|  71.306804443299|
+-----------+-----------------+



In [ ]:
# Exercice 2 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [ ]:
# 2. Session Spark Streaming
spark = SparkSession.builder.appName("SparkStreamingExample").getOrCreate()

In [ ]:
# 3. Schéma StructType du fichier CSV
schema = StructType([
    StructField("Date", StringType(), True),
    StructField("Open", DoubleType(), True),
    StructField("High", DoubleType(), True),
    StructField("Low", DoubleType(), True),
    StructField("Close", DoubleType(), True),
    StructField("Volume", DoubleType(), True),
    StructField("Adj Close", DoubleType(), True)
])


In [ ]:
# 4. Lecture des données à partir d'un répertoire HDFS
streaming_df = spark.readStream.option("header", "true").schema(schema).csv("hdfs://path/to/walmart_stock.csv")

In [ ]:
# 5. Filtre des données pour les colonnes Date, Open et Close
filtered_df = streaming_df.select("Date", "Open", "Close")

In [ ]:
# 6. Envoie des résultats sur la console
query = filtered_df.writeStream.outputMode("append").format("console").start()